##  Predict your coarse-grained genre 

In [2]:
import pandas as pd
import numpy as np
import ast

from sklearn.preprocessing import LabelEncoder, StandardScaler, normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from collections import Counter
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier



In [5]:

df = pd.read_csv("../data/intermediate/df_phase1.csv")
df_classif = pd.read_csv("../data/intermediate/df_classif_phase1.csv")
df_audio = pd.read_csv("../data/intermediate/df_audio_phase1.csv")


## 1. Taxonomie

In [3]:
# 0. Préparation des données

# On part de df_classif avec 'genre_top' et 'genres_all_names'

# Conversion en liste Python
if isinstance(df_classif["genres_all_names"].iloc[0], str):
    df_classif["genres_all_names"] = df_classif["genres_all_names"].apply(ast.literal_eval)

df_g = df_classif.copy()


# 1. Matrice genre_top x sous-genres

# Lister les sous-genres les plus fréquents
all_tags = [tag for sub in df_g["genres_all_names"] for tag in sub]
tag_counts = Counter(all_tags)

# Garde les 100 sous-genres les plus fréquents
top_tags = [t for t, c in tag_counts.most_common(100)]
tag_to_idx = {t: i for i, t in enumerate(top_tags)}

# Initialiser la matrice de co-occurrence
genres = sorted(df_g["genre_top"].unique())
genre_to_idx = {g: i for i, g in enumerate(genres)}

cooc = np.zeros((len(genres), len(top_tags)), dtype=float)

# Remplir la matrice : co-occurrence genre_top / sous-genre
for _, row in df_g[["genre_top", "genres_all_names"]].iterrows():
    gi = genre_to_idx[row["genre_top"]]
    for tag in row["genres_all_names"]:
        if tag in tag_to_idx:
            ti = tag_to_idx[tag]
            cooc[gi, ti] += 1

# Normalisation ligne : chaque genre devient un "profil" de sous-genres
cooc_norm = normalize(cooc, norm="l2", axis=1)

# 2. Réduction de dimension 

# SVD tronquée pour projeter dans un espace de petite dimension 
svd = TruncatedSVD(n_components=5, random_state=42)
cooc_reduced = svd.fit_transform(cooc_norm)


# 3. Clustering en 4 groupes coarse

kmeans_4 = KMeans(n_clusters=4, random_state=42, n_init="auto")
clusters_4 = kmeans_4.fit_predict(cooc_reduced)

taxo4 = pd.DataFrame({
    "genre_top": genres,
    "cluster_id_4": clusters_4
}).sort_values("cluster_id_4")

print("\nTaxonomie (4 groupes) :")
print(taxo4)



Taxonomie (4 groupes) :
              genre_top  cluster_id_4
1             Classical             0
5          Experimental             0
10                 Jazz             0
9         International             0
12                  Pop             1
11  Old-Time / Historic             1
4            Electronic             1
7               Hip-Hop             1
15               Spoken             1
2               Country             2
0                 Blues             3
3        Easy Listening             3
6                  Folk             3
8          Instrumental             3
13                 Rock             3
14             Soul-RnB             3


- On obtient en 4 groupes.

- Cluster 0 : Classical, Experimental, Jazz, International → famille “Art / Spécialisé”, des genres souvent plus instrumentaux, ou non‑mainstream.
- Cluster 1 : Pop, Old-Time / Historic, Electronic, Hip-Hop, Spoken → famille “Mainstream / Moderne”, centrée sur les musiques populaires et les formes parlées.
- Cluster 2 : Country isolé → profil de sous‑genres très spécifique (country/americana)
- Cluster 3 : Blues, Easy Listening, Folk, Instrumental, Rock, Soul-RnB → gros bloc “Rock / Roots & Soul”, mêlant rock, blues, folk et variantes plus douces/instrumentales.
​

In [ ]:
# Ajouter le cluster au df_classif via un merge
cluster_map = dict(zip(genres, clusters_4))  
df_classif["cluster_id_4"] = df_classif["genre_top"].map(cluster_map)

# Compter le nombre de morceaux par cluster
print(df_classif["cluster_id_4"].value_counts())


cluster_id_4
3    16697
1    15152
0    12621
2      163
Name: count, dtype: int64


- Le cluster 2 ne contient que 163 morceaux, alors que les autres en ont entre 12 621 et 16 697 : c’est une classe ultra‑minoritaire par rapport au reste.
- Avec si peu d’exemples, les modèles supervisés apprennent très mal ce groupe (rappel quasi nul), ce qui tire vers le bas la macro‑F1 même quand l’accuracy globale est bonne.
- Comme Country est musicalement proche de Folk / Blues / Rock (présents dans le cluster 3 “roots/rock/soul”), le choix raisonnable est de fusionner le cluster 2 avec le cluster 3 pour obtenir trois familles plus équilibrées.

In [16]:
coarse_map = {
    # Groupe 1 : Art / Spécialisé (cluster 0)
    "Classical": "Art/Spécialisé",
    "Experimental": "Art/Spécialisé",
    "Jazz": "Art/Spécialisé",
    "International": "Art/Spécialisé",

    # Groupe 2 : Mainstream (cluster 1)
    "Pop": "Mainstream",
    "Electronic": "Mainstream",
    "Hip-Hop": "Mainstream",
    "Spoken": "Mainstream",
    "Old-Time / Historic": "Mainstream",

    # Groupe 3 : Rock / Roots / Country & Soul (clusters 2 + 3)
    "Country": "Rock/Roots/Country&Soul",
    "Blues": "Rock/Roots/Country&Soul",
    "Easy Listening": "Rock/Roots/Country&Soul",
    "Folk": "Rock/Roots/Country&Soul",
    "Instrumental": "Rock/Roots/Country&Soul",
    "Rock": "Rock/Roots/Country&Soul",
    "Soul-RnB": "Rock/Roots/Country&Soul",
}

df_classif["genre_coarse"] = df_classif["genre_top"].map(coarse_map)
print(df_classif["genre_coarse"].value_counts(dropna=False))


genre_coarse
Rock/Roots/Country&Soul    16860
Mainstream                 15152
Art/Spécialisé             12621
Name: count, dtype: int64


## 2. Encodage de la cible et choix des features

On encode la nouvelle cible `genre_coarse` en entiers pour les modèles de classification,
et on sélectionne les mêmes features numériques que ceux choisi pour la task1 pour pouvoir comparer à la fin


In [17]:
# Encoder la cible coarse (4 classes)
le_coarse = LabelEncoder()
df_classif["genre_coarse_encoded"] = le_coarse.fit_transform(df_classif["genre_coarse"])

# Colonnes à exclure des features (cibles, id, listes de genres, texte)
cols_to_drop = [
    "track_id",
    "genre_top",
    "genre_encoded",
    "genre_coarse",
    "genre_coarse_encoded",
    "genres",
    "genres_all",
    "genres_names",
    "genres_all_names",
    "artist_name",
    "album_title",
    "title",
]

cols_to_drop = [c for c in cols_to_drop if c in df_classif.columns]

# On part de toutes les colonnes sauf celles à exclure
X_full = df_classif.drop(columns=cols_to_drop)

# On garde uniquement les colonnes numériques pour les modèles sklearn
feature_cols = X_full.select_dtypes(include=["int64", "float64"]).columns.tolist()

X = X_full[feature_cols].fillna(0)
y = df_classif["genre_coarse_encoded"]

print("Nb de features :", len(feature_cols))
print("Features utilisées :", feature_cols)
print("X shape :", X.shape, "| y shape :", y.shape)


Nb de features : 41
Features utilisées : ['album_tracks', 'artist_latitude', 'artist_longitude', 'duration', 'favorites', 'interest', 'listens', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'tempo', 'valence', 'spectral_bandwidth_kurtosis_01', 'spectral_bandwidth_max_01', 'spectral_bandwidth_mean_01', 'spectral_bandwidth_median_01', 'spectral_bandwidth_min_01', 'spectral_bandwidth_skew_01', 'spectral_bandwidth_std_01', 'spectral_centroid_kurtosis_01', 'spectral_centroid_max_01', 'spectral_centroid_mean_01', 'spectral_centroid_median_01', 'spectral_centroid_min_01', 'spectral_centroid_skew_01', 'spectral_centroid_std_01', 'spectral_rolloff_kurtosis_01', 'spectral_rolloff_max_01', 'spectral_rolloff_mean_01', 'spectral_rolloff_median_01', 'spectral_rolloff_min_01', 'spectral_rolloff_skew_01', 'spectral_rolloff_std_01', 'log_duration', 'log_listens', 'log_favorites', 'duration_min', 'favorites_per_listen']
X shape : (44633, 41) | y shape : (44633

## 3.Split train / validation / test

In [18]:
# Train+Val vs Test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train vs Val
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)  

print("Train :", X_train.shape, " Val :", X_val.shape, " Test :", X_test.shape)


Train : (26779, 41)  Val : (8927, 41)  Test : (8927, 41)


## 4. Entraînement de plusieurs modèles sur le set de validation

On teste plusieurs familles de modèles de classification :
- modèles linéaires (Logistic Regression),
- modèles à base de distances (kNN, SVM),
- modèles d'arbres (Decision Tree, Random Forest),
- modèles de boosting (Gradient Boosting, AdaBoost),
- gradient de boosting (XGBoost, LightGBM),
- réseaux de neurones (MLP)

In [19]:
models = {
    # Modèles qui ont besoin de scaling
    "LogReg": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=1000, n_jobs=-1))
    ]),
    "kNN": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier(n_neighbors=15))
    ]),
    "SVM_RBF": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf", C=3, gamma="scale"))
    ]),

    # Modèles arbres / boosting (pas besoin de scaler)
    "DecisionTree": DecisionTreeClassifier(max_depth=None, random_state=42),
    "RandomForest": RandomForestClassifier(
        n_estimators=300, max_depth=None, random_state=42, n_jobs=-1
    ),
    "GradBoost": GradientBoostingClassifier(random_state=42),
}

for name, model in models.items():
    print(f"\n=== {name} (validation) ===")
    # Apprentissage sur le train
    model.fit(X_train, y_train)
    # Prédictions sur le set de validation
    y_val_pred = model.predict(X_val)
    # Rapport de classification
    print(classification_report(y_val, y_val_pred, digits=3))



=== LogReg (validation) ===
              precision    recall  f1-score   support

           0      0.621     0.521     0.566      2524
           1      0.650     0.597     0.622      3031
           2      0.599     0.715     0.652      3372

    accuracy                          0.620      8927
   macro avg      0.623     0.611     0.614      8927
weighted avg      0.622     0.620     0.618      8927


=== kNN (validation) ===
              precision    recall  f1-score   support

           0      0.706     0.565     0.628      2524
           1      0.682     0.683     0.683      3031
           2      0.653     0.750     0.698      3372

    accuracy                          0.675      8927
   macro avg      0.681     0.666     0.669      8927
weighted avg      0.678     0.675     0.673      8927


=== SVM_RBF (validation) ===
              precision    recall  f1-score   support

           0      0.699     0.653     0.675      2524
           1      0.738     0.700     0.719 

- Accuracy globale : de 0.60 (DecisionTree) à 0.71–0.713 pour les meilleurs modèles (SVM RBF et RandomForest), avec kNN et GradBoost juste derrière autour de 0.68–0.69.
- Pour les 3 familles coarse, les F1‑scores restent raisonnables pour tous les modèles :

    LogReg ≈ 0.56–0.65,

    kNN ≈ 0.63–0.70,

    SVM / RandomForest montent à 0.69–0.72, donc ils séparent mieux les genres.
    ​

- DecisionTree est le moins bon : accuracy ≈ 0.60 et F1 < 0.62 sur toutes les classes, ce qui montre qu’un seul arbre sur‑ajuste un peu le train mais généralise moins bien.

- La macro‑avg et la weighted‑avg sont très proches pour tous les modèles (différence < 0.01–0.02), ce qui signifie que les trois classes sont apprises de façon assez équilibrée

In [20]:
models_boost = {
    # Réseau de neurones (besoin de scaling)
    "MLP": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", MLPClassifier(
            hidden_layer_sizes=(64, 32),
            activation="relu",
            max_iter=200,
            random_state=42
        )),
    ]),

    # Boosting classique
    "AdaBoost": AdaBoostClassifier(
        n_estimators=200,
        learning_rate=0.5,
        random_state=42,
    ),

    # Gradient boosting optimisés
    "XGBoost": XGBClassifier(
        n_estimators=300,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="multi:softmax",
        num_class=len(np.unique(y_train)),
        random_state=42,
        n_jobs=-1,
    ),
    "LightGBM": LGBMClassifier(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="multiclass",
        num_class=len(np.unique(y_train)),
        random_state=42,
        n_jobs=-1,
    ),
}

for name, model in models_boost.items():
    print(f"\n=== {name} (validation) ===")
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    print(classification_report(y_val, y_val_pred, digits=3))



=== MLP (validation) ===


c:\Users\Nada\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


              precision    recall  f1-score   support

           0      0.692     0.624     0.656      2524
           1      0.730     0.688     0.708      3031
           2      0.683     0.770     0.724      3372

    accuracy                          0.700      8927
   macro avg      0.702     0.694     0.696      8927
weighted avg      0.702     0.700     0.699      8927


=== AdaBoost (validation) ===
              precision    recall  f1-score   support

           0      0.657     0.471     0.548      2524
           1      0.627     0.627     0.627      3031
           2      0.582     0.706     0.638      3372

    accuracy                          0.613      8927
   macro avg      0.622     0.601     0.604      8927
weighted avg      0.618     0.613     0.609      8927


=== XGBoost (validation) ===
              precision    recall  f1-score   support

           0      0.771     0.717     0.743      2524
           1      0.779     0.762     0.771      3031
           2  

- MLP a une accuracy d’environ 0.70, avec des F1 par classe comprises entre 0.65 et 0.72; c’est un progrès
- AdaBoost reste en dessous, avec une accuracy ~0.61 et un F1 plus faible pour la classe 0 (0.55), ce qui confirme que, sur ce dataset, il est moins adapté que les autres.
- XGBoost et LightGBM donnent les meilleurs résultats, avec une accuracy de 0.764 et des F1 très proches pour les trois classes (≈ 0.74–0.78), d’où une macro‑F1 et une weighted‑F1 quasi identiques (~0.76), donc les trois familles de genres sont bien apprises de manière équilibrée.

### 5 Sélection des meilleurs modèles et évaluation finale

On sélectionne les 2 modèles les plus prometteurs (RandomForest, XGBoost et LightGBM).
On les réentraîne sur `Train + Val` pour utiliser un maximum de données, puis on mesure une seule fois la performance sur le set de test tenu de côté.


In [21]:
best_models = {
    "RandomForest": RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        random_state=42,
        n_jobs=-1
    ),
    "XGBoost": XGBClassifier(
        n_estimators=300,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="multi:softmax",
        num_class=len(np.unique(y_train)),
        random_state=42,
        n_jobs=-1,
    ),
    "LightGBM": LGBMClassifier(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="multiclass",
        num_class=len(np.unique(y_train)),
        random_state=42,
        n_jobs=-1,
    ),
}

X_train_full = pd.concat([X_train, X_val], axis=0)
y_train_full = pd.concat([y_train, y_val], axis=0)

# Évaluation finale sur le test
for name, model in best_models.items():
    print(f"\n=== {name} (test final) ===")
    model.fit(X_train_full, y_train_full)
    y_test_pred = model.predict(X_test)
    print(classification_report(y_test, y_test_pred, digits=3))


=== RandomForest (test final) ===
              precision    recall  f1-score   support

           0      0.718     0.655     0.685      2524
           1      0.725     0.723     0.724      3031
           2      0.717     0.765     0.740      3372

    accuracy                          0.720      8927
   macro avg      0.720     0.715     0.716      8927
weighted avg      0.720     0.720     0.719      8927


=== XGBoost (test final) ===
              precision    recall  f1-score   support

           0      0.774     0.708     0.739      2524
           1      0.781     0.775     0.778      3031
           2      0.761     0.815     0.787      3372

    accuracy                          0.771      8927
   macro avg      0.772     0.766     0.768      8927
weighted avg      0.771     0.771     0.770      8927


=== LightGBM (test final) ===
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014528 seconds.
You can set `force_col_wise=true` to r

- RandomForest obtient une accuracy de 0.72, avec des F1 entre 0.69 et 0.74 sur les trois familles
- XGBoost et LightGBM ont  autour de 0.77 d’accuracy sur le test, avec des F1 ≈ 0.74–0.79 pour chaque classe -> mieux que RandomForest
- les macro‑moyennes et weighted‑moyennes sont quasi identiques pour les trois modèles (≈ 0.77 pour XGBoost/LightGBM) -> performances homogènes entre genres coarse.​

## Comparaison avec genre_top

Sur Task 1 (genre_top, 16 classes), même les meilleurs modèles (XGBoost/LightGBM) montent à 0.74–0.77 d’accuracy, mais la F1 macro reste limitée (0.58–0.65) et la balanced accuracy plafonne vers 0.59, ce qui montre que certaines classes rares sont encore mal prédites.

Sur Task 2 (3 genres coarse), XGBoost/LightGBM gardent une accuracy similaire (~0.77) mais la macro‑F1 grimpe à ~0.76 et la perf est très homogène entre classes (macro ≈ weighted), donc le modèle ne sacrifie plus les classes minoritaires.